In [1]:
using StreamOps

In [ ]:
@macroexpand @pipeline OpReturn() OpPrint()

In [ ]:
using BenchmarkTools

function test_manual(count)
	last::Ref{Float64} = Ref(Float64(NaN))
	pipeline = function(x)
		x = abs(x)
		x = x * x
		tmp = last[]
		last[] = x
		tmp
	end
	current::Float64 = Float64(NaN)
	for i in 1:count
		current = pipeline(Float64(i))
	end
	current
end

function test_pipeline(count)
	pipeline = @pipeline OpAbs OpPrev{Float64} OpReturn
	current::Float64 = Float64(NaN)
	for i in 1:count
		current = pipeline(Float64(i))
	end
	current
end
display(@benchmark test_manual(1e6) samples = 30 evals = 3)
display(@benchmark test_pipeline(1e6) samples = 30 evals = 3)

In [ ]:
# using ProfileView
# ProfileView.@profview map(x -> test(1e7), 1:10)

# using Profile, ProfileSVG
# @Profile.profile test_pipeline(1e8)
# ProfileSVG.view(maxdepth=300, maxframes=3000)